# Analyzing CIA Factbook Data Using SQL

####  The data is from the  [CIA World Factbook](# https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. Here, I have used SQL in Jupyter Notebook to analyze data from the [factbook.db](# https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) database.

 Code to connect our Jupyter Notebook to our database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Query the database to get the information present in database

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Query to return first five information on the table facts in the database

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Query that returns the following:
Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [4]:
%%sql
SELECT MIN(population) AS Minimum_population, MAX(population) AS Maximum_population,
MIN(population_growth) AS Minimum_population_growth, MAX(population_growth) AS Maximum_population_growth
FROM facts

Done.


Minimum_population,Maximum_population,Minimum_population_growth,Maximum_population_growth
0,7256490011,0.0,4.02


query that returns the countries with the minimum population

In [5]:
%%sql
SELECT *
   FROM facts
 WHERE population = (SELECT MIN(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


query that returns the countries with the maximum population

In [6]:
%%sql
SELECT *
   FROM facts
 WHERE population = (SELECT MAX(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Query that returns the following excluding row for the whole world:
Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


calculate the average value for population and area

In [8]:
%%sql
SELECT AVG(population), AVG(area)
   FROM facts
  WHERE name <> 'World'; 

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


 query that finds all countries meeting both population is above average and
the area is below average.

In [9]:
%%sql
SELECT * 
   FROM facts 
 WHERE population > (SELECT AVG(population) FROM facts ) 
   AND 
    area < (SELECT AVG(area) FROM facts ) ;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Which country has the most people?

In [10]:
%%sql
SELECT name, population AS max_population_of_people FROM facts
 WHERE population = (SELECT MAX(population) FROM facts WHERE name <> 'World');

Done.


name,max_population_of_people
China,1367485388


Which country has the highest growth rate?

In [11]:
%%sql
SELECT name, population AS highest_growth_rate FROM facts
 WHERE population_growth = (SELECT MAX(population_growth) FROM facts WHERE name <> 'World');

Done.


name,highest_growth_rate
South Sudan,12042910


Which countries have the highest ratios of water to land?

In [12]:
%%sql
SELECT MAX(CAST(area_water/area_land AS FLOAT)) AS highest_ratio_of_water_to_land
FROM facts;

Done.


highest_ratio_of_water_to_land
905.0


Which countries have more water than land?

In [13]:
 %%sql
SELECT name AS countries_having_more_water_than_land, CAST(area_water/area_land AS FLOAT) AS Ratio  FROM facts WHERE(RATIO > 1);

Done.


countries_having_more_water_than_land,Ratio
British Indian Ocean Territory,905.0
Virgin Islands,4.0


Which countries will add the most people to their populations next year?

In [14]:
%%sql
SELECT name AS country_that_will_add_most_people_next_year FROM facts
WHERE population_growth = (SELECT MAX (population_growth) 
                           FROM facts WHERE name!= 'World');

Done.


country_that_will_add_most_people_next_year
South Sudan


Which countries have a higher death rate than birth rate?

In [15]:
%%sql
SELECT name AS countries_having_death_rate_higher_than_birth_rate, CAST(death_rate/birth_rate AS FLOAT) AS ratio_of_death_rate_by_birth_rate FROM facts
WHERE death_rate> birth_rate;

Done.


countries_having_death_rate_higher_than_birth_rate,ratio_of_death_rate_by_birth_rate
Austria,1.0010626992561105
Belarus,1.2485981308411216
Bosnia and Herzegovina,1.0992108229988726
Bulgaria,1.6188340807174888
Croatia,1.288888888888889
Czech Republic,1.0737279335410175
Estonia,1.1798287345385348
Germany,1.3482880755608027
Greece,1.2806004618937643
Hungary,1.3897379912663756


Which countries have the highest population/area ratio ?

In [16]:
%%sql
SELECT name, MAX(CAST(population/ area AS FLOAT)) AS population_by_area_ratio FROM facts;

Done.


name,population_by_area_ratio
Macau,21168.0
